<a href="https://colab.research.google.com/github/zachjoachim7/deep_learning_labs/blob/main/DL_Lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a
href="https://colab.research.google.com/github/wingated/cs474_labs_f2019/blob/master/DL_Lab6.ipynb"
  target="_parent">
  <img
    src="https://colab.research.google.com/assets/colab-badge.svg"
    alt="Open In Colab"/>
</a>

# Lab 6: Sequence-to-sequence models

### Description:
For this lab, you will code up the [char-rnn model of Karpathy](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). This is a recurrent neural network that is trained probabilistically on sequences of characters, and that can then be used to sample new sequences that are like the original.

This lab will help you develop several new skills, as well as understand some best practices needed for building large models. In addition, we'll be able to create networks that generate neat text!

### Deliverable:
- Fill in the code for the RNN (using PyTorch's built-in GRU).
- Fill in the training loop
- Fill in the evaluation loop. In this loop, rather than using a validation set, you will sample text from the RNN.
- Implement your own GRU cell.
- Train your RNN on a new domain of text (Star Wars, political speeches, etc. - have fun!)

### Grading Standards:
- 20% Implementation the RNN
- 20% Implementation training loop
- 20% Implementation of evaluation loop
- 20% Implementation of your own GRU cell
- 20% Training of your RNN on a domain of your choice

### Tips:
- Read through all the helper functions, run them, and make sure you understand what they are doing
- At each stage, ask yourself: What should the dimensions of this tensor be? Should its data type be float or int? (int is called `long` in PyTorch)
- Don't apply a softmax inside the RNN if you are using an nn.CrossEntropyLoss (this module already applies a softmax to its input).

### Example Output:
An example of my final samples are shown below (more detail in the
final section of this writeup), after 150 passes through the data.
Please generate about 15 samples for each dataset.

<code>
And ifte thin forgision forward thene over up to a fear not your
And freitions, which is great God. Behold these are the loss sub
And ache with the Lord hath bloes, which was done to the holy Gr
And appeicis arm vinimonahites strong in name, to doth piseling
And miniquithers these words, he commanded order not; neither sa
And min for many would happine even to the earth, to said unto m
And mie first be traditions? Behold, you, because it was a sound
And from tike ended the Lamanites had administered, and I say bi
</code>


---

## Part 0: Readings, data loading, and high level training

---

There is a tutorial here that will help build out scaffolding code, and get an understanding of using sequences in pytorch.

* Read the following

> * [Pytorch sequence-to-sequence tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html) (You will be implementing the decoder, not the encoder, as we are not doing sequence-to-sequence translation.)
> * [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)






In [1]:
! wget -O ./text_files.tar.gz 'https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz'
! tar -xzf text_files.tar.gz
! pip install unidecode
! pip install torch

import unidecode
import string
import random

all_characters = string.printable
n_characters = len(all_characters)
file = unidecode.unidecode(open('./text_files/lotr.txt').read())
file_len = len(file)
print('file_len =', file_len)

--2024-02-16 16:37:44--  https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz
Resolving piazza.com (piazza.com)... 23.22.156.213, 44.216.95.21, 35.174.203.227, ...
Connecting to piazza.com (piazza.com)|23.22.156.213|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-uploads.piazza.com/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz [following]
--2024-02-16 16:37:44--  https://cdn-uploads.piazza.com/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz
Resolving cdn-uploads.piazza.com (cdn-uploads.piazza.com)... 13.249.141.26, 13.249.141.11, 13.249.141.91, ...
Connecting to cdn-uploads.piazza.com (cdn-uploads.piazza.com)|13.249.141.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1533290 (1.5M) [application/x-gzip]
Saving to: ‘./text_files.tar.gz’

./text_files.tar.gz 100%[===================>]   1.46M  --.-KB/s

In [2]:
chunk_len = 200

def random_chunk():
  start_index = random.randint(0, file_len - chunk_len)
  end_index = start_index + chunk_len + 1
  return file[start_index:end_index]

print(random_chunk())

all the while I sit and think 
of times there were before, 

I listen for returning feet 
and voices at the door. 

It was a cold grey day near the end of December. The East Wind was 
streaming through


In [3]:
import torch
# Turn string into list of longs
def char_tensor(string):
  tensor = torch.zeros(len(string)).long()
  for c in range(len(string)):
      tensor[c] = all_characters.index(string[c])
  return tensor

print(char_tensor('abcDEF'))

tensor([10, 11, 12, 39, 40, 41])


---

## Part 4: Creating your own GRU cell

**(Come back to this later - its defined here so that the GRU will be defined before it is used)**

---

The cell that you used in Part 1 was a pre-defined Pytorch layer. Now, write your own GRU class using the same parameters as the built-in Pytorch class does.

Please do not look at the documentation's code for the GRU cell definition. The answer is right there in the code, and in theory, you could just cut-and-paste it. This bit is on your honor!

**TODO:**
* Create a custom GRU cell

**DONE:**



In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class GRU(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers):
    super(GRU, self).__init__()
    self.num_layers = num_layers
    # W_ir and W_hr
    self.W_ir = nn.ModuleList([nn.Linear(input_size, hidden_size) for _ in range(num_layers)])
    self.W_hr = nn.ModuleList([nn.Linear(input_size, hidden_size) for _ in range(num_layers)])
    # W_iz and W_hz
    self.W_iz = nn.ModuleList([nn.Linear(input_size, hidden_size) for _ in range(num_layers)])
    self.W_hz = nn.ModuleList([nn.Linear(input_size, hidden_size) for _ in range(num_layers)])
    # W_in and W_hn
    self.W_in = nn.ModuleList([nn.Linear(input_size, hidden_size) for _ in range(num_layers)])
    self.W_hn = nn.ModuleList([nn.Linear(input_size, hidden_size) for _ in range(num_layers)])
    # Activation Functions
    self.sigmoid = nn.Sigmoid()
    self.tanh = nn.Tanh()


  def forward(self, inputs, hidden):
    # Each layer does the following:
    # r_t = sigmoid(W_ir*x_t + b_ir + W_hr*h_(t-1) + b_hr)
    # z_t = sigmoid(W_iz*x_t + b_iz + W_hz*h_(t-1) + b_hz)
    # n_t = tanh(W_in*x_t + b_in + r_t**(W_hn*h_(t-1) + b_hn))
    # h_(t) = (1 - z_t)**n_t + z_t**h_(t-1)
    # Where ** is hadamard product (not matrix multiplication, but elementwise multiplication)

    hiddens = torch.tensor.zeros(hidden.shape).cuda()
    for i in range(self.num_layers):
      hidden_input = hidden[i]
      r_t = self.sigmoid(self.W_ir[i](inputs) + self.W_hr[i](hidden_input))
      z_t = self.sigmoid(self.W_iz[i](inputs) + self.W_hz[i](hidden_input))
      n_t = self.tanh(self.W_in[i](inputs) + self.W_hn[i](hidden_input))
      h_t = torch.mul((1-z_t), n_t) + torch.mul(z_t, hidden_input)
      del r_t, z_t, n_t

      hiddens[i] = h_t
      outputs = h_t

    return outputs, hiddens

---

##  Part 1: Building a sequence to sequence model

---

Great! We have the data in a useable form. We can switch out which text file we are reading from, and trying to simulate.

We now want to build out an RNN model, in this section, we will use all built in Pytorch pieces when building our RNN class.


**TODO:**
* Create an RNN class that extends from nn.Module.

**DONE:**



In [5]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, n_layers=1):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.n_layers = n_layers

    # more stuff here...
    self.embedding = nn.Embedding(output_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
    self.out = nn.Linear(hidden_size, output_size)

  def forward(self, input_char, hidden):
    # by reviewing the documentation, construct a forward function that properly uses the output
    # of the GRU

    # stuff here
    output = self.embedding(input_char)

    if len(output.size()) == 1:
      output = output.unsqueeze(0).unsqueeze(0)
    elif len(output.size()) == 2:
      output = output.unsqueeze(0)

    output = F.relu(output)
    output, hidden = self.gru(output, hidden)
    out_decoded = self.out(output).squeeze(0)

    return out_decoded, hidden

  def init_hidden(self):
    return torch.zeros(self.n_layers, 1, self.hidden_size)

In [6]:
def random_training_set():
  chunk = random_chunk()
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp, target

---

## Part 2: Sample text and Training information

---

We now want to be able to train our network, and sample text after training.

This function outlines how training a sequence style network goes.

**TODO:**
* Fill in the pieces.

**DONE:**




In [7]:
# NOTE: decoder_optimizer, decoder, and criterion will be defined below as global variables
def train(inp, target):
  ## initialize hidden layers, set up gradient and loss
    # your code here
  ## /

  decoder_optimizer.zero_grad()
  hidden = decoder.init_hidden()
  loss = 0

  # more stuff here...
  for i in range(len(inp)):
    y_hat, hidden = decoder(inp[i], hidden)
    loss += criterion(y_hat, target[i].unsqueeze(0))
  loss.backward()
  decoder_optimizer.step()

  return loss

---

## Part 3: Sample text and Training information

---

You can at this time, if you choose, also write out your train loop boilerplate that samples random sequences and trains your RNN. This will be helpful to have working before writing your own GRU class.

If you are finished training, or during training, and you want to sample from the network you may consider using the following function. If your RNN model is instantiated as `decoder`then this will probabilistically sample a sequence of length `predict_len`

**TODO:**
* Fill out the evaluate function to generate text frome a primed string

**DONE:**



In [8]:
def sample_outputs(output, temperature):
    """Takes in a vector of unnormalized probability weights and samples a character from the distribution"""
    # As temperature approaches 0, this sampling function becomes argmax (no randomness)
    # As temperature approaches infinity, this sampling function becomes a purely random choice
    return torch.multinomial(torch.exp(output / temperature), 1)

def evaluate(prime_str='A', predict_len=100, temperature=0.8):
  ## initialize hidden state, initialize other useful variables
    # your code here
  ## /
  hidden = decoder.init_hidden()

  # This loop primes the output and the hidden layers with the char_tensor function
    # defined above and the prime_str that is present in the function parameters.
  for char in prime_str:
    output, hidden = decoder(char_tensor(char), hidden)

  output_list = []
  # This loop is very similar to the training.
  for i in range(predict_len):
    # You take a 'predicted' character index
    character_index = sample_outputs(output, temperature)
    # Send that index into the decoder (which is just the RNN model), and gives us the actual input
      # and the hidden.
    output, hidden = decoder(character_index, hidden)
    # Appends those values to a list that is returned with the primer.
    output_list.append(all_characters[character_index])

  return prime_str + ''.join(output_list)


---

## Part 4: (Create a GRU cell, requirements above)

---



---

## Part 5: Run it and generate some text!

---


**TODO:**
* Create some cool output

**DONE:**




Assuming everything has gone well, you should be able to run the main function in the scaffold code, using either your custom GRU cell or the built in layer, and see output something like this. I trained on the “lotr.txt” dataset, using chunk_length=200, hidden_size=100 for 2000 epochs. These are the results, along with the prime string:

---

 G:

 Gandalf was decrond.
'All have lord you. Forward the road at least walk this is stuff, and
went to the long grey housel-winding and kindled side was a sleep pleasuring, I do long
row hrough. In  

 lo:

 lost death it.
'The last of the gatherings and take you,' said Aragorn, shining out of the Gate.
'Yes, as you there were remembaused to seen their pass, when? What
said here, such seven an the sear

 lo:

 low, and frod to keepn
Came of their most. But here priced doubtless to an Sam up is
masters; he left hor as they are looked. And he could now the long to stout in the right fro horseless of
the like

 I:

 I had been the
in his eyes with the perushed to lest, if then only the ring and the legended
of the less of the long they which as the
enders of Orcovered and smood, and the p

 I:

 I they were not the lord of the hoomes.
Home already well from the Elves. And he sat strength, and we
housed out of the good of the days to the mountains from his perith.

'Yess! Where though as if  

 Th:

 There yarden
you would guard the hoor might. Far and then may was
croties, too began to see the drumbred many line
and was then hoard walk and they heart, and the chair of the
Ents of way, might was

 G:

 Gandalf
been lat of less the round of the stump; both and seemed to the trees and perished they
lay are speered the less; and the wind the steep and have to she
precious. There was in the oonly went

 wh:

 which went out of the door.
Hull the King and of the The days of his brodo
stumbler of the windard was a thing there, then it been shining langing
to him poor land. They hands; though they seemed ou

 ra:

 rather,' have all the least deather
down of the truven beginning to the house of sunk.
'Nark shorts of the Eyes of the Gate your great nothing as Eret.
'I wander trust horn, and there were not, it  

 I:

 I can have no mind
together! Where don't may had one may little blung
terrible to tales. And turn and Gandalf shall be not to as only the Cattring
not stopped great the out them forms. On they she lo

---


In [10]:
import time
n_epochs = 5000
print_every = 200
plot_every = 10
hidden_size = 200
n_layers = 3
lr = 0.001

decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

In [16]:
n_epochs = 1001
for epoch in range(1, n_epochs + 1):
  loss_ = train(*random_training_set())
  loss_avg += loss_

  if epoch % print_every == 0:
      print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
      print(evaluate('Wh', 100), '\n')

  if epoch % plot_every == 0:
      all_losses.append(loss_avg / plot_every)
      loss_avg = 0

[111.39304280281067 (200 19%) 557.5963]
Whed c
Rriud alt ad. od wow
meng incaand ttot beow ho tcere thot asd Lot cnor agrrhe sociut the teom,  

[204.05555701255798 (400 39%) 446.0882]
Whor sbamill the 

vead Ame caar oct was weris mond be ooes that ell and we said nos the gallafedeld 
 

[296.3060305118561 (600 59%) 397.2455]
Whing you I 
light, hi haid and the bame. 
Whe lirn.' 

'pinibut shanred yut and the sall thim, the So 

[388.7120954990387 (800 79%) 361.0090]
Whing pirsing on them liped peathas do to qleath a passe mittle in I both to dent 
a main the up are t 

[483.46568608283997 (1000 99%) 334.3880]
Whey himsen up from 
bive treat and they reptands of they me to was were unore the broughts in the 
am 



In [17]:
for i in range(10):
  start_strings = [" Th", " wh", " he", " I ", " ca", " G", " lo", " ra"]
  start = random.randint(0,len(start_strings)-1)
  print(start_strings[start])
#   all_characters.index(string[c])
  print(evaluate(start_strings[start], 200), '\n')

 lo
 looking in their gisile to is 
that pistegs brope unought would 
fell all be neanturss far the in lear wooder in there e spe the some hands and the hide asy it to that into the can of The noors to grist 

 lo
 lork, and that are 
all the ronuhed as 
and wimis are will 
sleak to that his well to they spemt, and mbat besiled 
the spower pate. 

Net shadcard and're them. his all be thas a mong to ever of qut the 

 he
 he botty there meard of Hisped. 

But whem a shat it hands of the could 
for mide to the spoin 
store. 

Has away to it all think on Mess. Not in the caknes; and it. 

And is the 
meem. The wore be verk 

 Th
 The Gudden of the Enping his stills geep of the Gond 






lit of the 
was the 
do this of the Et of the 
fear and store 
and nothly on the Elves of they move the Enes of the counse and my looked. 'For 

 ra
 randing the brey. The ling and apping the End.' 

'He more that grats of the mester of the world in evains as has Frodo the Ellikis; by a ring and the clea

---

## Part 6: Generate output on a different dataset

---

**TODO:**

* Choose a textual dataset. Here are some [text datasets](https://www.kaggle.com/datasets?tags=14104-text+data%2C13205-text+mining) from Kaggle

* Generate some decent looking results and evaluate your model's performance (say what it did well / not so well)

**DONE:**



In [10]:
import time
n_characters = len(string.printable)
file = unidecode.unidecode(open('./text_files/tiny_shakespeare.txt').read())
file_len = len(file)
print('file_len =', file_len)

n_epochs = 1001
print_every = 200
plot_every = 10
hidden_size = 200
n_layers = 3
lr = 0.001

decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
  loss_ = train(*random_training_set())
  loss_avg += loss_

  if epoch % print_every == 0:
      print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
      print(evaluate('Wh', 100), '\n')

  if epoch % plot_every == 0:
      all_losses.append(loss_avg / plot_every)
      loss_avg = 0

for i in range(10):
  start_strings = [" Th", " wh", " he", " I ", " ca", " G", " lo", " ra"]
  start = random.randint(0,len(start_strings)-1)
  print(start_strings[start])
  print(evaluate(start_strings[start], 200), '\n')

file_len = 1115394
[93.88283085823059 (200 19%) 494.6338]
Whos hat miy thov wor,
That ten mind ik hemerir ir time toy to kay on bes thal bou, bont mar ant you p 

[185.69856429100037 (400 39%) 408.5297]
Why foit bere noe the in thang't is son ssargt lome.

LINCINLE:
CIARCINEA:
Neas:
Bod, tor totast steat 

[281.3085227012634 (600 59%) 409.3501]
When the spomef
Ouse with the a'luss afally
Bith and heere.

GRICHHoOd the shellood.

YeRIPINIET:
E'd! 

[382.4420771598816 (800 79%) 376.1064]
Whis wither marest,
What so samy, in the lip's
Outh the forther'd stard honous it.

MORVER:
His his li 

[477.03542947769165 (1000 99%) 380.5674]
Whow his can to shall,
Sarebours him ever his boly me be and thou-hing but thank him thy truth than an 

 ra
 raigs,
And he hour betumn dean friet be all swe all high list, neqfen'd
Wis and but me age bather,
And but in kniair hone the lords, and have were, that thou put
Thy by his nenven, purmide,
And hory our 

 lo
 lords.

LUCCASS:
Yet him, where hath my be 